### Import Required Libraries and Set Up Environment Variables

In [3]:
# Dependencies
import requests
import time
from dotenv import load_dotenv
import os
import pandas as pd
import json
from pandas import json_normalize

In [5]:
# Set environment variables from the .env in the local environment
load_dotenv()

nyt_api_key = os.getenv("NYT_API_KEY")
tmdb_api_key = os.getenv("TMDB_API_KEY")

### Access the New York Times API

In [8]:
# Set the base URL
url = "https://api.nytimes.com/svc/search/v2/articlesearch.json?"

# Filter for movie reviews with "love" in the headline
# section_name should be "Movies"
# type_of_material should be "Review"
filter_query = 'section_name:"Movies" AND type_of_material:"Review" AND headline:"love"'

# Use a sort filter, sort by newest
sort = "newest"

# Select the following fields to return:
# headline, web_url, snippet, source, keywords, pub_date, byline, word_count
field_list = "headline,web_url,snippet,source,keywords,pub_date,byline,word_count"

# Search for reviews published between a begin and end date
begin_date = "20130101"
end_date = "20230531"

# Build URL
query_url = (
    f"{url}api-key={nyt_api_key}&begin_date={begin_date}&end_date={end_date}"
    + f'&fq={filter_query}&sort={sort}&fl={field_list}'
)

In [10]:
# Create an empty list to store the reviews
reviews_list = []

# loop through pages 0-19
for page in range(20):
    # create query with a page number
    # API results show 10 articles at a time
    query_url = (
        f"{url}api-key={nyt_api_key}&begin_date={begin_date}&end_date={end_date}"
        + f"&fq={filter_query}&sort={sort}&fl={field_list}&page={page}"
    )

    
    # Make a "GET" request and retrieve the JSON
    response = requests.get(query_url)
    
    # Add a twelve second interval between queries to stay within API query limits
    time.sleep(12)
    
    # Try and save the reviews to the reviews_list
    try:
        response_json = response.json()
        
        reviews = response_json.get('response', {}).get('docs', [])

        # If no results, print the page number and break from the loop
        if not reviews:
            print(f"No results on page {page}. Breaking the loop.")
            break
            
        # loop through the reviews["response"]["docs"] and append each review to the list
        for review in reviews:
            reviews_list.append({
                "headline": review["headline"]["main"],
                "web_url": review["web_url"],
                "snippet": review["snippet"],
                "source": review["source"],
                "keywords": review["keywords"],
                "pub_date": review["pub_date"],
                "byline": review["byline"],
                "word_count": review["word_count"]
            })

        # Print the page that was just retrieved
        print(f"Checked page {page}")
    # Print the page number that had no results then break from the loop
    except Exception as e:
        print(f"Error on page {page}: {e}")
        break


Checked page 0
Checked page 1
Checked page 2
Checked page 3
Checked page 4
Checked page 5
Checked page 6
Checked page 7
Checked page 8
Checked page 9
Checked page 10
Checked page 11
Checked page 12
Checked page 13
Checked page 14
Checked page 15
Checked page 16
Checked page 17
Checked page 18
Checked page 19


In [11]:
# Preview the first 5 results in JSON format
# Use json.dumps with argument indent=4 to format data
print(json.dumps(reviews_list[:5], indent=4))

[
    {
        "headline": "\u2018The Attachment Diaries\u2019 Review: Love, Sick",
        "web_url": "https://www.nytimes.com/2023/05/25/movies/the-attachment-diaries-review.html",
        "snippet": "A gynecologist and her patient form a horrifyingly twisted connection in this batty, bloody Argentine melodrama.",
        "source": "The New York Times",
        "keywords": [
            {
                "name": "subject",
                "value": "Movies",
                "rank": 1,
                "major": "N"
            },
            {
                "name": "creative_works",
                "value": "The Attachment Diaries (Movie)",
                "rank": 2,
                "major": "N"
            },
            {
                "name": "persons",
                "value": "Diment, Valentin Javier",
                "rank": 3,
                "major": "N"
            }
        ],
        "pub_date": "2023-05-25T11:00:03+0000",
        "byline": {
            "original": "By 

In [12]:
# Convert reviews_list to a Pandas DataFrame using json_normalize()
df_reviews = json_normalize(reviews_list)
df_reviews


,headline,web_url,snippet,source,keywords,pub_date,word_count,byline.original,byline.person,byline.organization
0,"‘The Attachment Diaries’ Review: Love, Sick",https://www.nytimes.com/2023/05/25/movies/the-...,A gynecologist and her patient form a horrifyi...,The New York Times,"[{'name': 'subject', 'value': 'Movies', 'rank'...",2023-05-25T11:00:03+0000,295,By Jeannette Catsoulis,"[{'firstname': 'Jeannette', 'middlename': None...",None
1,Review: ‘What’s Love Got to Do With It?’ Proba...,https://www.nytimes.com/2023/05/04/movies/what...,Two childhood friends navigate cultural differ...,The New York Times,"[{'name': 'subject', 'value': 'Movies', 'rank'...",2023-05-04T17:16:45+0000,287,By Jeannette Catsoulis,"[{'firstname': 'Jeannette', 'middlename': None...",None
2,‘You Can Live Forever’ Review: Do You Love Me ...,https://www.nytimes.com/2023/05/04/movies/you-...,Religion comes between two girls falling in lo...,The New York Times,"[{'name': 'subject', 'value': 'Movies', 'rank'...",2023-05-04T11:00:08+0000,294,By Elisabeth Vincentelli,"[{'firstname': 'Elisabeth', 'middlename': None...",None
3,‘A Tourist’s Guide to Love’ Review: A Wearying...,https://www.nytimes.com/2023/04/21/movies/a-to...,Rachael Leigh Cook stars in this bland rom-com...,The New York Times,"[{'name': 'subject', 'value': 'Movies', 'rank'...",2023-04-21T07:03:25+0000,276,By Elisabeth Vincentelli,"[{'firstname': 'Elisabeth', 'middlename': None...",None
4,‘Other People’s Children’ Review: True Romance,https://www.nytimes.com/2023/04/20/movies/othe...,A radiant Virginie Efira stars as a Parisian t...,The New York Times,"[{'name': 'subject', 'value': 'Movies', 'rank'...",2023-04-20T15:35:13+0000,801,By Manohla Dargis,"[{'firstname': 'Manohla', 'middlename': None, ...",None
...,...,...,...,...,...,...,...,...,...,...
195,Review: A Combustible Pair Find Love in ‘The O...,https://www.nytimes.com/2017/03/09/movies/the-...,This moody romance stars Tatiana Maslany (“Orp...,The New York Times,"[{'name': 'subject', 'value': 'Movies', 'rank'...",2017-03-09T21:54:58+0000,251,By Andy Webster,"[{'firstname': 'Andy', 'middlename': None, 'la...",None
196,"Review: Love as the World Wars, in ‘The Ottoma...",https://www.nytimes.com/2017/03/09/movies/revi...,A nurse travels to the Ottoman Empire on the e...,The New York Times,"[{'name': 'subject', 'value': 'Movies', 'rank'...",2017-03-09T21:53:12+0000,267,By Neil Genzlinger,"[{'firstname': 'Neil', 'middlename': None, 'la...",None
197,Review: It’s All Mirth and Taxes in ‘Love & Ta...,https://www.nytimes.com/2017/03/02/movies/love...,Josh Kornbluth runs afoul of the Internal Reve...,The New York Times,"[{'name': 'creative_works', 'value': 'Love & T...",2017-03-02T21:44:18+0000,246,By Ken Jaworowski,"[{'firstname': 'Ken', 'middlename': None, 'las...",None
198,"Review: ‘Everybody Loves Somebody,’ a Rom-Com ...",https://www.nytimes.com/2017/02/16/movies/ever...,A messed-up heroine is asked to choose between...,The New York Times,"[{'name': 'subject', 'value': 'Movies', 'rank'...",2017-02-16T21:45:50+0000,256,By Jeannette Catsoulis,"[{'firstname': 'Jeannette', 'middlename': None...",None


In [13]:
# Extract the title from the "headline.main" column and
# save it to a new column "title"
# Title is between unicode characters \u2018 and \u2019. 
# End string should include " Review" to avoid cutting title early

df_reviews['title'] = df_reviews['headline'].apply(
    lambda x: x[x.find('\u2018') + 1:x.rfind('\u2019')] if '\u2018' in x and '\u2019' in x else None
)

df_reviews


,headline,web_url,snippet,source,keywords,pub_date,word_count,byline.original,byline.person,byline.organization,title
0,"‘The Attachment Diaries’ Review: Love, Sick",https://www.nytimes.com/2023/05/25/movies/the-...,A gynecologist and her patient form a horrifyi...,The New York Times,"[{'name': 'subject', 'value': 'Movies', 'rank'...",2023-05-25T11:00:03+0000,295,By Jeannette Catsoulis,"[{'firstname': 'Jeannette', 'middlename': None...",None,The Attachment Diaries
1,Review: ‘What’s Love Got to Do With It?’ Proba...,https://www.nytimes.com/2023/05/04/movies/what...,Two childhood friends navigate cultural differ...,The New York Times,"[{'name': 'subject', 'value': 'Movies', 'rank'...",2023-05-04T17:16:45+0000,287,By Jeannette Catsoulis,"[{'firstname': 'Jeannette', 'middlename': None...",None,What’s Love Got to Do With It?
2,‘You Can Live Forever’ Review: Do You Love Me ...,https://www.nytimes.com/2023/05/04/movies/you-...,Religion comes between two girls falling in lo...,The New York Times,"[{'name': 'subject', 'value': 'Movies', 'rank'...",2023-05-04T11:00:08+0000,294,By Elisabeth Vincentelli,"[{'firstname': 'Elisabeth', 'middlename': None...",None,You Can Live Forever
3,‘A Tourist’s Guide to Love’ Review: A Wearying...,https://www.nytimes.com/2023/04/21/movies/a-to...,Rachael Leigh Cook stars in this bland rom-com...,The New York Times,"[{'name': 'subject', 'value': 'Movies', 'rank'...",2023-04-21T07:03:25+0000,276,By Elisabeth Vincentelli,"[{'firstname': 'Elisabeth', 'middlename': None...",None,A Tourist’s Guide to Love
4,‘Other People’s Children’ Review: True Romance,https://www.nytimes.com/2023/04/20/movies/othe...,A radiant Virginie Efira stars as a Parisian t...,The New York Times,"[{'name': 'subject', 'value': 'Movies', 'rank'...",2023-04-20T15:35:13+0000,801,By Manohla Dargis,"[{'firstname': 'Manohla', 'middlename': None, ...",None,Other People’s Children
...,...,...,...,...,...,...,...,...,...,...,...
195,Review: A Combustible Pair Find Love in ‘The O...,https://www.nytimes.com/2017/03/09/movies/the-...,This moody romance stars Tatiana Maslany (“Orp...,The New York Times,"[{'name': 'subject', 'value': 'Movies', 'rank'...",2017-03-09T21:54:58+0000,251,By Andy Webster,"[{'firstname': 'Andy', 'middlename': None, 'la...",None,The Other Half
196,"Review: Love as the World Wars, in ‘The Ottoma...",https://www.nytimes.com/2017/03/09/movies/revi...,A nurse travels to the Ottoman Empire on the e...,The New York Times,"[{'name': 'subject', 'value': 'Movies', 'rank'...",2017-03-09T21:53:12+0000,267,By Neil Genzlinger,"[{'firstname': 'Neil', 'middlename': None, 'la...",None,The Ottoman Lieutenant
197,Review: It’s All Mirth and Taxes in ‘Love & Ta...,https://www.nytimes.com/2017/03/02/movies/love...,Josh Kornbluth runs afoul of the Internal Reve...,The New York Times,"[{'name': 'creative_works', 'value': 'Love & T...",2017-03-02T21:44:18+0000,246,By Ken Jaworowski,"[{'firstname': 'Ken', 'middlename': None, 'las...",None,Love & Taxes
198,"Review: ‘Everybody Loves Somebody,’ a Rom-Com ...",https://www.nytimes.com/2017/02/16/movies/ever...,A messed-up heroine is asked to choose between...,The New York Times,"[{'name': 'subject', 'value': 'Movies', 'rank'...",2017-02-16T21:45:50+0000,256,By Jeannette Catsoulis,"[{'firstname': 'Jeannette', 'middlename': None...",None,"Everybody Loves Somebody,"


In [14]:
# Extract 'name' and 'value' from items in "keywords" column
def extract_keywords(keyword_list):
    extracted_keywords = ""
    for item in keyword_list:
        # Extract 'name' and 'value'
        keyword = f"{item['name']}: {item['value']};" 
        # Append the keyword item to the extracted_keywords list
        extracted_keywords += keyword
    return extracted_keywords

# Fix the "keywords" column by converting cells from a list to a string
df_reviews['keywords'] = df_reviews['keywords'].apply(lambda x: extract_keywords(x) if isinstance(x, list) else x)
df_reviews

,headline,web_url,snippet,source,keywords,pub_date,word_count,byline.original,byline.person,byline.organization,title
0,"‘The Attachment Diaries’ Review: Love, Sick",https://www.nytimes.com/2023/05/25/movies/the-...,A gynecologist and her patient form a horrifyi...,The New York Times,subject: Movies;creative_works: The Attachment...,2023-05-25T11:00:03+0000,295,By Jeannette Catsoulis,"[{'firstname': 'Jeannette', 'middlename': None...",None,The Attachment Diaries
1,Review: ‘What’s Love Got to Do With It?’ Proba...,https://www.nytimes.com/2023/05/04/movies/what...,Two childhood friends navigate cultural differ...,The New York Times,"subject: Movies;persons: Kapur, Shekhar;person...",2023-05-04T17:16:45+0000,287,By Jeannette Catsoulis,"[{'firstname': 'Jeannette', 'middlename': None...",None,What’s Love Got to Do With It?
2,‘You Can Live Forever’ Review: Do You Love Me ...,https://www.nytimes.com/2023/05/04/movies/you-...,Religion comes between two girls falling in lo...,The New York Times,subject: Movies;creative_works: You Can Live F...,2023-05-04T11:00:08+0000,294,By Elisabeth Vincentelli,"[{'firstname': 'Elisabeth', 'middlename': None...",None,You Can Live Forever
3,‘A Tourist’s Guide to Love’ Review: A Wearying...,https://www.nytimes.com/2023/04/21/movies/a-to...,Rachael Leigh Cook stars in this bland rom-com...,The New York Times,subject: Movies;creative_works: A Tourist's Gu...,2023-04-21T07:03:25+0000,276,By Elisabeth Vincentelli,"[{'firstname': 'Elisabeth', 'middlename': None...",None,A Tourist’s Guide to Love
4,‘Other People’s Children’ Review: True Romance,https://www.nytimes.com/2023/04/20/movies/othe...,A radiant Virginie Efira stars as a Parisian t...,The New York Times,"subject: Movies;persons: Zlotowski, Rebecca;cr...",2023-04-20T15:35:13+0000,801,By Manohla Dargis,"[{'firstname': 'Manohla', 'middlename': None, ...",None,Other People’s Children
...,...,...,...,...,...,...,...,...,...,...,...
195,Review: A Combustible Pair Find Love in ‘The O...,https://www.nytimes.com/2017/03/09/movies/the-...,This moody romance stars Tatiana Maslany (“Orp...,The New York Times,subject: Movies;creative_works: The Other Half...,2017-03-09T21:54:58+0000,251,By Andy Webster,"[{'firstname': 'Andy', 'middlename': None, 'la...",None,The Other Half
196,"Review: Love as the World Wars, in ‘The Ottoma...",https://www.nytimes.com/2017/03/09/movies/revi...,A nurse travels to the Ottoman Empire on the e...,The New York Times,subject: Movies;creative_works: The Ottoman Li...,2017-03-09T21:53:12+0000,267,By Neil Genzlinger,"[{'firstname': 'Neil', 'middlename': None, 'la...",None,The Ottoman Lieutenant
197,Review: It’s All Mirth and Taxes in ‘Love & Ta...,https://www.nytimes.com/2017/03/02/movies/love...,Josh Kornbluth runs afoul of the Internal Reve...,The New York Times,creative_works: Love & Taxes (Movie);persons: ...,2017-03-02T21:44:18+0000,246,By Ken Jaworowski,"[{'firstname': 'Ken', 'middlename': None, 'las...",None,Love & Taxes
198,"Review: ‘Everybody Loves Somebody,’ a Rom-Com ...",https://www.nytimes.com/2017/02/16/movies/ever...,A messed-up heroine is asked to choose between...,The New York Times,subject: Movies;creative_works: Everybody Love...,2017-02-16T21:45:50+0000,256,By Jeannette Catsoulis,"[{'firstname': 'Jeannette', 'middlename': None...",None,"Everybody Loves Somebody,"


In [15]:
# Create a list from the "title" column using to_list()
titles_list = df_reviews['title'].dropna().to_list()

# Display the titles list
titles_list

['The Attachment Diaries',
 'What’s Love Got to Do With It?',
 'You Can Live Forever',
 'A Tourist’s Guide to Love',
 'Other People’s Children',
 'One True Loves',
 'The Lost Weekend: A Love Story',
 'A Thousand and One',
 'Your Place or Mine',
 'Love in the Time of Fentanyl',
 'Pamela, a Love Story',
 'In From the Side',
 'After Love',
 'Alcarràs',
 'Nelly & Nadine',
 'Lady Chatterley’s Lover',
 'The Sound of Christmas',
 'The Inspection',
 'Bones and All',
 'My Policeman',
 'About Fate’ Review: Love the One You',
 'Waiting for Bojangles',
 'I Love My Dad',
 'A Love Song',
 'Alone Together',
 'Art of Love',
 'The Wheel',
 'Thor: Love and Thunder’ Review: A God',
 'Both Sides of the Blade',
 'Fire of Love',
 'Love & Gelato',
 'Stay Prayed Up',
 'Benediction’ Review: A Poet',
 'Dinner in America',
 'In a New York Minute',
 'Anaïs in Love',
 'I Love America',
 'See You Then',
 'La Mami',
 'Love After Love',
 'Deep Water',
 'Lucy and Desi',
 'Cyrano',
 'The In Between',
 'Book of Love',
 

In [16]:
# Create a list from the "title" column using to_list()
# These titles will be used in the query for The Movie Database
titles_list = df_reviews['title'].dropna().to_list()
titles_list

['The Attachment Diaries',
 'What’s Love Got to Do With It?',
 'You Can Live Forever',
 'A Tourist’s Guide to Love',
 'Other People’s Children',
 'One True Loves',
 'The Lost Weekend: A Love Story',
 'A Thousand and One',
 'Your Place or Mine',
 'Love in the Time of Fentanyl',
 'Pamela, a Love Story',
 'In From the Side',
 'After Love',
 'Alcarràs',
 'Nelly & Nadine',
 'Lady Chatterley’s Lover',
 'The Sound of Christmas',
 'The Inspection',
 'Bones and All',
 'My Policeman',
 'About Fate’ Review: Love the One You',
 'Waiting for Bojangles',
 'I Love My Dad',
 'A Love Song',
 'Alone Together',
 'Art of Love',
 'The Wheel',
 'Thor: Love and Thunder’ Review: A God',
 'Both Sides of the Blade',
 'Fire of Love',
 'Love & Gelato',
 'Stay Prayed Up',
 'Benediction’ Review: A Poet',
 'Dinner in America',
 'In a New York Minute',
 'Anaïs in Love',
 'I Love America',
 'See You Then',
 'La Mami',
 'Love After Love',
 'Deep Water',
 'Lucy and Desi',
 'Cyrano',
 'The In Between',
 'Book of Love',
 

### Access The Movie Database API

In [18]:
# Prepare The Movie Database query
url = "https://api.themoviedb.org/3/search/movie?query="
tmdb_key_string = "&api_key=" + tmdb_api_key

In [38]:
# Create an empty list to store the results
tmdb_movies_list = []

# Create a request counter to sleep the requests after a multiple of 50 requests
request_counter = 0

# Prepare The Movie Database query
tmdb_search_url = "https://api.themoviedb.org/3/search/movie?query="
tmdb_key_string = "&api_key=" + tmdb_api_key

# Loop through the titles
for title in titles_list:
    # Check if we need to sleep before making a request
    if request_counter > 0 and request_counter % 50 == 0:
        print("Sleeping for 10 seconds to avoid hitting rate limits.")
        time.sleep(10)

    # Add 1 to the request counter
    request_counter += 1
    
    # Perform a "GET" request for The Movie Database
    search_url = f"{tmdb_search_url}{title}{tmdb_key_string}"
    search_response = requests.get(search_url)
    
    # Include a try clause to search for the full movie details
    try:
        search_data = search_response.json()
        if search_data['results']:
            # Get movie id
            movie_id = search_data['results'][0]['id']

            # Make a request for the full movie details
            details_url = f"https://api.themoviedb.org/3/movie/{movie_id}?api_key={tmdb_api_key}"
            details_response = requests.get(details_url)
            details_data = details_response.json()
            
            # Extract the genre names into a list
            genres = [genre['name'] for genre in details_data.get('genres', [])]
            
            # Extract the spoken_languages' English name into a list
            languages = [language['english_name'] for language in details_data.get('spoken_languages', [])]
            
            # Extract the production_countries' name into a list
            countries = [country['name'] for country in details_data.get('production_countries', [])]
            
            # Add the relevant data to a dictionary and append it to the tmdb_movies_list list
            tmdb_movies_list.append({
                'title': title,
                'id': movie_id,
                'genres': genres,
                'languages': languages,
                'countries': countries,
                'release_date': details_data.get('release_date', ''),
                'runtime': details_data.get('runtime', 0),
                'overview': details_data.get('overview', '')
            })
            
            # Print out the title that was found
            print(f"Found details for title: {title}")
        else:
            print(f"No results found for title: {title}")
    except Exception as e:
        print(f"Error fetching details for title '{title}': {e}")


Found details for title: The Attachment Diaries
Found details for title: What’s Love Got to Do With It?
Found details for title: You Can Live Forever
Found details for title: A Tourist’s Guide to Love
Found details for title: Other People’s Children
Found details for title: One True Loves
Found details for title: The Lost Weekend: A Love Story
Found details for title: A Thousand and One
Found details for title: Your Place or Mine
Found details for title: Love in the Time of Fentanyl
Found details for title: Pamela, a Love Story
Found details for title: In From the Side
Found details for title: After Love
Found details for title: Alcarràs
Found details for title: Nelly & Nadine
Found details for title: Lady Chatterley’s Lover
Found details for title: The Sound of Christmas
Found details for title: The Inspection
Found details for title: Bones and All
Found details for title: My Policeman
No results found for title: About Fate’ Review: Love the One You
Found details for title: Waiting fo

In [36]:
# Preview the first 5 results in JSON format
# Use json.dumps with argument indent=4 to format data
print(json.dumps(tmdb_movies_list[:5], indent=4))

[
    {
        "title": "The Attachment Diaries",
        "id": 743040,
        "genres": [
            "Drama",
            "Mystery",
            "Thriller",
            "Horror"
        ],
        "languages": [
            "Spanish"
        ],
        "countries": [
            "Argentina"
        ],
        "release_date": "2021-10-07",
        "runtime": 102,
        "overview": "Argentina, 1970s. A desperate young woman goes to a clinic to have a clandestine abortion. As her pregnancy is already through the fourth month, the doctor refuses. Instead, she proposes to sell the baby to one of her clients and offers to provide shelter in her house until the child is born. Their disturbed personalities will become intertwined in a strange and dangerous relationship."
    },
    {
        "title": "What\u2019s Love Got to Do With It?",
        "id": 800301,
        "genres": [
            "Romance",
            "Comedy"
        ],
        "languages": [
            "English",
        

In [42]:
# Convert the TMDB movies list to a DataFrame
df_tmdb_movies = pd.DataFrame(tmdb_movies_list)
df_tmdb_movies

,title,id,genres,languages,countries,release_date,runtime,overview
0,The Attachment Diaries,743040,"[Drama, Mystery, Thriller, Horror]",[Spanish],[Argentina],2021-10-07,102,"Argentina, 1970s. A desperate young woman goes..."
1,What’s Love Got to Do With It?,800301,"[Romance, Comedy]","[English, Portuguese, Urdu]","[France, United Kingdom]",2023-01-26,109,Two childhood friends now in their thirties mu...
2,You Can Live Forever,887580,"[Drama, Romance]","[English, French]","[Canada, United States of America]",2023-03-24,96,"When Jaime, a gay teenager, is sent to live in..."
3,A Tourist’s Guide to Love,813726,"[Romance, Comedy]","[English, Vietnamese]",[United States of America],2023-04-21,96,"After an unexpected break up, a travel executi..."
4,Other People’s Children,803694,"[Drama, Comedy]","[French, English]",[France],2022-09-21,104,"Rachel loves her life, her students, her frien..."
...,...,...,...,...,...,...,...,...
181,"Frantz,",377263,"[History, Drama, Romance]","[German, French]","[France, Germany]",2016-09-07,113,"In the aftermath of WWI, a young German who gr..."
182,The Other Half,381015,"[Drama, Romance]",[English],[Canada],2016-12-02,103,A grief-stricken man and a bipolar woman fall ...
183,The Ottoman Lieutenant,433356,"[Romance, Drama, War]","[Turkish, English]","[Turkey, United States of America]",2017-03-10,111,"Lillie, a determined American woman, ventures ..."
184,Love & Taxes,1248795,"[Romance, Action, Drama]",[Turkish],[Turkey],2024-03-13,99,After learning that the art thief she has been...


### Merge and Clean the Data for Export

In [46]:
# Merge the New York Times reviews and TMDB DataFrames on title
merged_df = pd.merge(df_reviews, df_tmdb_movies, on='title', how='inner')
merged_df

,headline,web_url,snippet,source,keywords,pub_date,word_count,byline.original,byline.person,byline.organization,title,id,genres,languages,countries,release_date,runtime,overview
0,"‘The Attachment Diaries’ Review: Love, Sick",https://www.nytimes.com/2023/05/25/movies/the-...,A gynecologist and her patient form a horrifyi...,The New York Times,subject: Movies;creative_works: The Attachment...,2023-05-25T11:00:03+0000,295,By Jeannette Catsoulis,"[{'firstname': 'Jeannette', 'middlename': None...",None,The Attachment Diaries,743040,"[Drama, Mystery, Thriller, Horror]",[Spanish],[Argentina],2021-10-07,102,"Argentina, 1970s. A desperate young woman goes..."
1,Review: ‘What’s Love Got to Do With It?’ Proba...,https://www.nytimes.com/2023/05/04/movies/what...,Two childhood friends navigate cultural differ...,The New York Times,"subject: Movies;persons: Kapur, Shekhar;person...",2023-05-04T17:16:45+0000,287,By Jeannette Catsoulis,"[{'firstname': 'Jeannette', 'middlename': None...",None,What’s Love Got to Do With It?,800301,"[Romance, Comedy]","[English, Portuguese, Urdu]","[France, United Kingdom]",2023-01-26,109,Two childhood friends now in their thirties mu...
2,‘You Can Live Forever’ Review: Do You Love Me ...,https://www.nytimes.com/2023/05/04/movies/you-...,Religion comes between two girls falling in lo...,The New York Times,subject: Movies;creative_works: You Can Live F...,2023-05-04T11:00:08+0000,294,By Elisabeth Vincentelli,"[{'firstname': 'Elisabeth', 'middlename': None...",None,You Can Live Forever,887580,"[Drama, Romance]","[English, French]","[Canada, United States of America]",2023-03-24,96,"When Jaime, a gay teenager, is sent to live in..."
3,‘A Tourist’s Guide to Love’ Review: A Wearying...,https://www.nytimes.com/2023/04/21/movies/a-to...,Rachael Leigh Cook stars in this bland rom-com...,The New York Times,subject: Movies;creative_works: A Tourist's Gu...,2023-04-21T07:03:25+0000,276,By Elisabeth Vincentelli,"[{'firstname': 'Elisabeth', 'middlename': None...",None,A Tourist’s Guide to Love,813726,"[Romance, Comedy]","[English, Vietnamese]",[United States of America],2023-04-21,96,"After an unexpected break up, a travel executi..."
4,‘Other People’s Children’ Review: True Romance,https://www.nytimes.com/2023/04/20/movies/othe...,A radiant Virginie Efira stars as a Parisian t...,The New York Times,"subject: Movies;persons: Zlotowski, Rebecca;cr...",2023-04-20T15:35:13+0000,801,By Manohla Dargis,"[{'firstname': 'Manohla', 'middlename': None, ...",None,Other People’s Children,803694,"[Drama, Comedy]","[French, English]",[France],2022-09-21,104,"Rachel loves her life, her students, her frien..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
183,"Review: ‘Frantz,’ a Mysterious Frenchman and t...",https://www.nytimes.com/2017/03/14/movies/fran...,François Ozon’s adaptation of Ernst Lubitsch’s...,The New York Times,"creative_works: Frantz (Movie);persons: Niney,...",2017-03-14T21:14:24+0000,708,By Stephen Holden,"[{'firstname': 'Stephen', 'middlename': None, ...",None,"Frantz,",377263,"[History, Drama, Romance]","[German, French]","[France, Germany]",2016-09-07,113,"In the aftermath of WWI, a young German who gr..."
184,Review: A Combustible Pair Find Love in ‘The O...,https://www.nytimes.com/2017/03/09/movies/the-...,This moody romance stars Tatiana Maslany (“Orp...,The New York Times,subject: Movies;creative_works: The Other Half...,2017-03-09T21:54:58+0000,251,By Andy Webster,"[{'firstname': 'Andy', 'middlename': None, 'la...",None,The Other Half,381015,"[Drama, Romance]",[English],[Canada],2016-12-02,103,A grief-stricken man and a bipolar woman fall ...
185,"Review: Love as the World Wars, in ‘The Ottoma...",https://www.nytimes.com/2017/03/09/movies/revi...,A nurse travels to the Ottoman Empire on the e...,The New York Times,subject: Movies;creative_works: The Ottoman Li...,2017-03-09T21:53:12+0000,267,By Neil Genzlinger,"[{'firstname': 'Neil', 'middlename': None, 'la...",None,The Ottoman Lieut

In [56]:
# Remove list brackets and quotation marks on the columns containing lists
# Create a list of the columns that need fixing
columns_to_fix = ['genres', 'languages', 'countries', 'byline.person']

# Create a list of characters to remove
characters_to_remove = ['[', ']', "'", '{', '}', ':']

def remove_characters(string, characters):
    for char in characters:
        string = string.replace(char, '')
    return string
# Loop through the list of columns to fix
for column in columns_to_fix:
    # Convert the column to type 'str'
    merged_df[column] = merged_df[column].astype(str)
    # Loop through characters to remove
    merged_df[column] = merged_df[column].apply(lambda x: remove_characters(x, characters_to_remove))


# Display the fixed DataFrame
merged_df



,headline,web_url,snippet,source,keywords,pub_date,word_count,byline.original,byline.person,byline.organization,title,id,genres,languages,countries,release_date,runtime,overview
0,"‘The Attachment Diaries’ Review: Love, Sick",https://www.nytimes.com/2023/05/25/movies/the-...,A gynecologist and her patient form a horrifyi...,The New York Times,subject: Movies;creative_works: The Attachment...,2023-05-25T11:00:03+0000,295,By Jeannette Catsoulis,"firstname Jeannette, middlename None, lastname...",None,The Attachment Diaries,743040,"Drama, Mystery, Thriller, Horror",Spanish,Argentina,2021-10-07,102,"Argentina, 1970s. A desperate young woman goes..."
1,Review: ‘What’s Love Got to Do With It?’ Proba...,https://www.nytimes.com/2023/05/04/movies/what...,Two childhood friends navigate cultural differ...,The New York Times,"subject: Movies;persons: Kapur, Shekhar;person...",2023-05-04T17:16:45+0000,287,By Jeannette Catsoulis,"firstname Jeannette, middlename None, lastname...",None,What’s Love Got to Do With It?,800301,"Romance, Comedy","English, Portuguese, Urdu","France, United Kingdom",2023-01-26,109,Two childhood friends now in their thirties mu...
2,‘You Can Live Forever’ Review: Do You Love Me ...,https://www.nytimes.com/2023/05/04/movies/you-...,Religion comes between two girls falling in lo...,The New York Times,subject: Movies;creative_works: You Can Live F...,2023-05-04T11:00:08+0000,294,By Elisabeth Vincentelli,"firstname Elisabeth, middlename None, lastname...",None,You Can Live Forever,887580,"Drama, Romance","English, French","Canada, United States of America",2023-03-24,96,"When Jaime, a gay teenager, is sent to live in..."
3,‘A Tourist’s Guide to Love’ Review: A Wearying...,https://www.nytimes.com/2023/04/21/movies/a-to...,Rachael Leigh Cook stars in this bland rom-com...,The New York Times,subject: Movies;creative_works: A Tourist's Gu...,2023-04-21T07:03:25+0000,276,By Elisabeth Vincentelli,"firstname Elisabeth, middlename None, lastname...",None,A Tourist’s Guide to Love,813726,"Romance, Comedy","English, Vietnamese",United States of America,2023-04-21,96,"After an unexpected break up, a travel executi..."
4,‘Other People’s Children’ Review: True Romance,https://www.nytimes.com/2023/04/20/movies/othe...,A radiant Virginie Efira stars as a Parisian t...,The New York Times,"subject: Movies;persons: Zlotowski, Rebecca;cr...",2023-04-20T15:35:13+0000,801,By Manohla Dargis,"firstname Manohla, middlename None, lastname D...",None,Other People’s Children,803694,"Drama, Comedy","French, English",France,2022-09-21,104,"Rachel loves her life, her students, her frien..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
183,"Review: ‘Frantz,’ a Mysterious Frenchman and t...",https://www.nytimes.com/2017/03/14/movies/fran...,François Ozon’s adaptation of Ernst Lubitsch’s...,The New York Times,"creative_works: Frantz (Movie);persons: Niney,...",2017-03-14T21:14:24+0000,708,By Stephen Holden,"firstname Stephen, middlename None, lastname H...",None,"Frantz,",377263,"History, Drama, Romance","German, French","France, Germany",2016-09-07,113,"In the aftermath of WWI, a young German who gr..."
184,Review: A Combustible Pair Find Love in ‘The O...,https://www.nytimes.com/2017/03/09/movies/the-...,This moody romance stars Tatiana Maslany (“Orp...,The New York Times,subject: Movies;creative_works: The Other Half...,2017-03-09T21:54:58+0000,251,By Andy Webster,"firstname Andy, middlename None, lastname Webs...",None,The Other Half,381015,"Drama, Romance",English,Canada,2016-12-02,103,A grief-stricken man and a bipolar woman fall ...
185,"Review: Love as the World Wars, in ‘The Ottoma...",https://www.nytimes.com/2017/03/09/movies/revi...,A nurse travels to the Ottoman Empire on the e...,The New York Times,subject: Movies;creative_works: The Ottoman Li...,2017-03-09T21:53:12+0000,267,By Neil Genzlinger,"firstname Neil, middlename None, lastname Genz...",None,The Ottoman Lieutenant,433356,"Romance, Drama, War","Turkis

In [58]:
# Drop "byline.person" column
merged_drop_df = merged_df.drop(columns=['byline.person'])
merged_drop_df

,headline,web_url,snippet,source,keywords,pub_date,word_count,byline.original,byline.organization,title,id,genres,languages,countries,release_date,runtime,overview
0,"‘The Attachment Diaries’ Review: Love, Sick",https://www.nytimes.com/2023/05/25/movies/the-...,A gynecologist and her patient form a horrifyi...,The New York Times,subject: Movies;creative_works: The Attachment...,2023-05-25T11:00:03+0000,295,By Jeannette Catsoulis,None,The Attachment Diaries,743040,"Drama, Mystery, Thriller, Horror",Spanish,Argentina,2021-10-07,102,"Argentina, 1970s. A desperate young woman goes..."
1,Review: ‘What’s Love Got to Do With It?’ Proba...,https://www.nytimes.com/2023/05/04/movies/what...,Two childhood friends navigate cultural differ...,The New York Times,"subject: Movies;persons: Kapur, Shekhar;person...",2023-05-04T17:16:45+0000,287,By Jeannette Catsoulis,None,What’s Love Got to Do With It?,800301,"Romance, Comedy","English, Portuguese, Urdu","France, United Kingdom",2023-01-26,109,Two childhood friends now in their thirties mu...
2,‘You Can Live Forever’ Review: Do You Love Me ...,https://www.nytimes.com/2023/05/04/movies/you-...,Religion comes between two girls falling in lo...,The New York Times,subject: Movies;creative_works: You Can Live F...,2023-05-04T11:00:08+0000,294,By Elisabeth Vincentelli,None,You Can Live Forever,887580,"Drama, Romance","English, French","Canada, United States of America",2023-03-24,96,"When Jaime, a gay teenager, is sent to live in..."
3,‘A Tourist’s Guide to Love’ Review: A Wearying...,https://www.nytimes.com/2023/04/21/movies/a-to...,Rachael Leigh Cook stars in this bland rom-com...,The New York Times,subject: Movies;creative_works: A Tourist's Gu...,2023-04-21T07:03:25+0000,276,By Elisabeth Vincentelli,None,A Tourist’s Guide to Love,813726,"Romance, Comedy","English, Vietnamese",United States of America,2023-04-21,96,"After an unexpected break up, a travel executi..."
4,‘Other People’s Children’ Review: True Romance,https://www.nytimes.com/2023/04/20/movies/othe...,A radiant Virginie Efira stars as a Parisian t...,The New York Times,"subject: Movies;persons: Zlotowski, Rebecca;cr...",2023-04-20T15:35:13+0000,801,By Manohla Dargis,None,Other People’s Children,803694,"Drama, Comedy","French, English",France,2022-09-21,104,"Rachel loves her life, her students, her frien..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
183,"Review: ‘Frantz,’ a Mysterious Frenchman and t...",https://www.nytimes.com/2017/03/14/movies/fran...,François Ozon’s adaptation of Ernst Lubitsch’s...,The New York Times,"creative_works: Frantz (Movie);persons: Niney,...",2017-03-14T21:14:24+0000,708,By Stephen Holden,None,"Frantz,",377263,"History, Drama, Romance","German, French","France, Germany",2016-09-07,113,"In the aftermath of WWI, a young German who gr..."
184,Review: A Combustible Pair Find Love in ‘The O...,https://www.nytimes.com/2017/03/09/movies/the-...,This moody romance stars Tatiana Maslany (“Orp...,The New York Times,subject: Movies;creative_works: The Other Half...,2017-03-09T21:54:58+0000,251,By Andy Webster,None,The Other Half,381015,"Drama, Romance",English,Canada,2016-12-02,103,A grief-stricken man and a bipolar woman fall ...
185,"Review: Love as the World Wars, in ‘The Ottoma...",https://www.nytimes.com/2017/03/09/movies/revi...,A nurse travels to the Ottoman Empire on the e...,The New York Times,subject: Movies;creative_works: The Ottoman Li...,2017-03-09T21:53:12+0000,267,By Neil Genzlinger,None,The Ottoman Lieutenant,433356,"Romance, Drama, War","Turkish, English","Turkey, United States of America",2017-03-10,111,"Lillie, a determined American woman, ventures ..."
186,Review: It’s All Mirth and Taxes in ‘Love & Ta...,https://www.nytimes.com/2017/03/02/movies/love...,Josh Kornbluth runs afoul of the Internal Reve...,The New York Times,creative_works: Love & Taxes (Movie);persons: ...,2017-03-02T21:44:18+0000,246,By Ken Jaworowski,None,Love & Taxes,1248795,"Romance, Action, Drama"

In [68]:
# Delete duplicate rows and reset index
merged_dd_df = merged_df.drop_duplicates()

# Reset the index
final_list_df = merged_dd_df.reset_index(drop=True)
final_list_df

,headline,web_url,snippet,source,keywords,pub_date,word_count,byline.original,byline.person,byline.organization,title,id,genres,languages,countries,release_date,runtime,overview
0,"‘The Attachment Diaries’ Review: Love, Sick",https://www.nytimes.com/2023/05/25/movies/the-...,A gynecologist and her patient form a horrifyi...,The New York Times,subject: Movies;creative_works: The Attachment...,2023-05-25T11:00:03+0000,295,By Jeannette Catsoulis,"firstname Jeannette, middlename None, lastname...",None,The Attachment Diaries,743040,"Drama, Mystery, Thriller, Horror",Spanish,Argentina,2021-10-07,102,"Argentina, 1970s. A desperate young woman goes..."
1,Review: ‘What’s Love Got to Do With It?’ Proba...,https://www.nytimes.com/2023/05/04/movies/what...,Two childhood friends navigate cultural differ...,The New York Times,"subject: Movies;persons: Kapur, Shekhar;person...",2023-05-04T17:16:45+0000,287,By Jeannette Catsoulis,"firstname Jeannette, middlename None, lastname...",None,What’s Love Got to Do With It?,800301,"Romance, Comedy","English, Portuguese, Urdu","France, United Kingdom",2023-01-26,109,Two childhood friends now in their thirties mu...
2,‘You Can Live Forever’ Review: Do You Love Me ...,https://www.nytimes.com/2023/05/04/movies/you-...,Religion comes between two girls falling in lo...,The New York Times,subject: Movies;creative_works: You Can Live F...,2023-05-04T11:00:08+0000,294,By Elisabeth Vincentelli,"firstname Elisabeth, middlename None, lastname...",None,You Can Live Forever,887580,"Drama, Romance","English, French","Canada, United States of America",2023-03-24,96,"When Jaime, a gay teenager, is sent to live in..."
3,‘A Tourist’s Guide to Love’ Review: A Wearying...,https://www.nytimes.com/2023/04/21/movies/a-to...,Rachael Leigh Cook stars in this bland rom-com...,The New York Times,subject: Movies;creative_works: A Tourist's Gu...,2023-04-21T07:03:25+0000,276,By Elisabeth Vincentelli,"firstname Elisabeth, middlename None, lastname...",None,A Tourist’s Guide to Love,813726,"Romance, Comedy","English, Vietnamese",United States of America,2023-04-21,96,"After an unexpected break up, a travel executi..."
4,‘Other People’s Children’ Review: True Romance,https://www.nytimes.com/2023/04/20/movies/othe...,A radiant Virginie Efira stars as a Parisian t...,The New York Times,"subject: Movies;persons: Zlotowski, Rebecca;cr...",2023-04-20T15:35:13+0000,801,By Manohla Dargis,"firstname Manohla, middlename None, lastname D...",None,Other People’s Children,803694,"Drama, Comedy","French, English",France,2022-09-21,104,"Rachel loves her life, her students, her frien..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
181,"Review: ‘Frantz,’ a Mysterious Frenchman and t...",https://www.nytimes.com/2017/03/14/movies/fran...,François Ozon’s adaptation of Ernst Lubitsch’s...,The New York Times,"creative_works: Frantz (Movie);persons: Niney,...",2017-03-14T21:14:24+0000,708,By Stephen Holden,"firstname Stephen, middlename None, lastname H...",None,"Frantz,",377263,"History, Drama, Romance","German, French","France, Germany",2016-09-07,113,"In the aftermath of WWI, a young German who gr..."
182,Review: A Combustible Pair Find Love in ‘The O...,https://www.nytimes.com/2017/03/09/movies/the-...,This moody romance stars Tatiana Maslany (“Orp...,The New York Times,subject: Movies;creative_works: The Other Half...,2017-03-09T21:54:58+0000,251,By Andy Webster,"firstname Andy, middlename None, lastname Webs...",None,The Other Half,381015,"Drama, Romance",English,Canada,2016-12-02,103,A grief-stricken man and a bipolar woman fall ...
183,"Review: Love as the World Wars, in ‘The Ottoma...",https://www.nytimes.com/2017/03/09/movies/revi...,A nurse travels to the Ottoman Empire on the e...,The New York Times,subject: Movies;creative_works: The Ottoman Li...,2017-03-09T21:53:12+0000,267,By Neil Genzlinger,"firstname Neil, middlename None, lastname Genz...",None,The Ottoman Lieutenant,433356,"Romance, Drama, War","Turkis

In [70]:
# Export data to CSV without the index
final_list_df.to_csv('final_list.csv', index=False)
